# 多线程和并发

## 执行代码的时间控制
clojure提供了一些函数可以控制代码的执行。

**delay**

delay可以让代码延迟执行，被delay包裹的代码只有在`deref`或者被 **`@`** 的时候才能执行。这看起来和执行一个函数没什么两样，但是delay的一个好处就是它能够**缓存代码的执行结果**，不需要重复进行计算，比如如下代码。值得注意的是，如果代码里面有副作用，比如println，在重复执行的时候，这个副作用也会被忽略。

可以通过`realized?`检查代码是否被解引用

In [11]:
(def d (delay (println "Running") :done!))
(deref d)
(deref d)
@d
(realized? d)

Running


#'user/d :done! :done! :done! true

**future**

future是clojure中启动一个线程的函数。对于future的调用会立即返回，执行结果保存在future当中，可以通过解引用来获得其值，如果值还没有获得，那么会阻塞直到获得结果。future也会保存代码的返回值。和delay不同的是它可以指定一个超时值。注意设定超时只能使用`deref`的方式，不能使用`@`。

In [16]:
(deref (future (Thread/sleep 5000) :done!) 1000 :impatient!)

:impatient!

**promise**

promise和之前的future/delay有些不同，它是定义的一个对象，你可以通过`deliver`向它传递值。

In [1]:
(def p (promise))
(realized? p)
(deliver p 42)
(realized? p)
@p

#'user/p false #object[clojure.core$promise$reify__7005 0xa88a8b3 {:status :ready, :val 42}] true 42

## 并发支持的特殊数据类型

对于其他编程语言，比如`python`,`ruby`，对一个对象的操作不是线程安全的。比如在python中，假如有一个类`Person`，实例化了一个对象`a`，当他从未婚状态转入到已婚状态的时候：

```
a.marry_status = "married"
a.spouse = "some one"
```

如果在更改status之后，设置spouse之前有另外一个线程读取这个对象，那么它处于一个不一致的状态。

在clojure中，由于`symbol`是不可以被更改的，clojure使用了一些其他特殊数据类型来进行线程安全的操作。

### atom

我们可以定义一个atom数据，然后使用 `@aotm-var`来获得这个数据的一个快照。这样的话我们并不会更改原来的的数据，而是直接产生了一个新的数据状态。对atom的更改使用`swap!`函数.

In [9]:
(def my-atom (atom {:name "woosley" :gender "male" :age 30}))
(let [old-woosley @my-atom]
    (swap! my-atom update-in [:age] inc)
    (swap! my-atom update-in [:age] inc)

    (do (println (str "old-woosley: " old-woosley))
        (println (str "new-woosley: " @my-atom))))

old-woosley: {:name "woosley", :gender "male", :age 30}
new-woosley: {:name "woosley", :gender "male", :age 32}


#'user/my-atom nil

`swap`使用一种`compare-and-set`的方式保证这个操作是线程安全的。

- 首先读取atom的数据a0
- 应用swap!的函数，获得结果b0
- 再次读取atom的数据，a1，
- 如果此时a0 = a1，返回b0
- 如果此时a0 != a1，使用a1的值重新应用`swap!`的函数。

这个流程保证了不同线程对同一个atom的修改是安全的。

`reset!`函数能够彻底重置一个atom

In [10]:
(reset! my-atom {:name "cloned woosley" :gender "male" :age "unknown"})

{:name "cloned woosley", :gender "male", :age "unknown"}

### ref
atom一般用在修改单个数据对象的时候，ref使用在需要同时修改多个数据对象的情况下，它使用`dosync`函数保证对一个transaction里面所有ref的`alter`操作都具有像数据库里面一样的ACI性质。`dosync`的内部修改在事务外部都是不可见的。

In [5]:
(def my-cookie (ref 10))
(def his-cookie (ref 0))
(defn give-my-cookie-to-him 
[] 
(dosync 
  (alter my-cookie - 1)
  (alter his-cookie + 1)
  (println (str @my-cookie " " @his-cookie))))
(give-my-cookie-to-him)

(def counter (ref 0))
(future (dosync
  (alter counter inc)
  (println (str "Inside counter " @counter "\n")
  (Thread/sleep 250)
  (alter counter inc)
  (println (str "counter " @counter)))))
  
(Thread/sleep 200)
(println (str "Outside counter " @counter)) ; can not see any dosync changes before dosync is committed

9 1
Outside counter 0


#'user/my-cookie #'user/his-cookie #'user/give-my-cookie-to-him nil #'user/counter #object[clojure.core$future_call$reify__6962 0x46b16feb {:status :pending, :val nil}] nil nil

和atom一样，在一个transaction/dosync结束的时候，如果发现里面的ref数据在外部被修改，会使用新的值重新运行一遍整个transaction。如果确认数据在外部是安全的，可以使用`commute`函数，这时候clojure不会关注ref是否被修改，放弃重试，直接commit。在这种情况下必须要保证没有外部修改。以下是一个使用`commute`不安全的例子。

In [11]:
(def receiver-a (ref #{}))
(def receiver-b (ref #{}))
(def giver (ref #{1}))
(do (future (dosync (let [gift (first @giver)]
                      (Thread/sleep 10)
                      (commute receiver-a conj gift) ;receiver A gets a gift
                      (commute giver disj gift))))
    (future (dosync (let [gift (first @giver)]
                      (Thread/sleep 50)
                      (commute receiver-b conj gift) ;receiver B gets a gift
                      (commute giver disj gift)))))

#'user/receiver-a #'user/receiver-b #'user/giver #object[clojure.core$future_call$reify__6962 0x28cfd608 {:status :pending, :val nil}]

In [12]:
@receiver-b
@receiver-a
@giver

#{1} #{1} #{}

显然一份礼物不同同时给两个人，这里由于使用了commute，导致了这个错误。

### var
通过`def`定义的符号都是`var`。
#### 动态var
- 我们通过`:dynamic`定义动态var，
- 一般此时使用如`*var-name*`之类的命名方式。
- 我们通过binding来改变var的值
- 通过binding改变的值只能存在于binding作用域中。

In [16]:
(def ^:dynamic *address* "dobby@elf.org")
(binding [*address* "woosley@gmail.com"]
    *address*)
*address*

#'user/*address* "woosley@gmail.com" "dobby@elf.org"

dynamic var一般用来在函数外部更改一些使用在函数里面的全局变量，clojure本身用到了许多dynamic var，比如`*out*`，下面是一个通过`bind *out*`改变println的输出的例子。

In [19]:
*out*
(binding [*out* (clojure.java.io/writer, "aaa.txt")]
    (println "write to file, not stdout"))
(slurp "aaa.txt")

#object[java.io.PrintWriter 0x5552071 "java.io.PrintWriter@5552071"] nil "write to file, not stdout\n"

`set!`函数可以更改一个var的值，但是注意它不能修改root binding。

In [22]:
(def ^:dynamic aaa 1)
(binding [*aaa* nil]
(set! *aaa* 2)
(println *aaa*))